# T09P02 - MdSM

# CASO DE ESTUDIO 2: Tiempo crítico de despeje de una falta trifásica en el punto medio de la línea B (de un conjunto de dos líneas en paralelo) y que se despeja por apertura de la misma.

## Enunciado

Determine el ángulo crítico de despeje de una falta trifásica en el punto medio de la línea B de un grupo de dos líneas en paralelo, y que se despeja por apertura de la misma.

Tome como datos:

- Potencia nominal del generador ($S_g$): 100 MVA.
- Tensión nominal en bornes del generador ($U_g$): 15 kV.
- Constante de inercia del generador ($H_g$): 3 s (Referida a la potencia del generador).
- Carga del generador previa a la falta: 100% potencia nominal.
- Factor de potencia del generador previo a la falta: $0.8$ inductivo.
- Reactancia transitoria del generador ($X'_g$): 0.3 pu.
- Potencia del transformador ($S_t$): 100 MVA.
- Reactancia de cc del transformador ($X_t$): 0.15 pu.
- Reactancia de la línea A ($X_{lA}$): 0.20 pu.
- Reactancia de la línea B ($X_{lB}$): 0.20 pu.
- Tensión del nudo de conexión ($U_{red}$): 220 kV.
- Potencia del nudo de conexión ($S_{red}$): infinita.

**Sistema eléctrico:**

<img src="./Figs/T09P02_Fig00.png" width=95%>

## Resolución

In [ ]:
# Importar librerías genéricas:
import numpy as np
import cmath as cm
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import scipy.integrate as integrate
import scipy.optimize as optim
import sympy

# Formato general de salida de resultados numéricos:
np.set_printoptions(precision=3, suppress=True)

### Datos del problema

In [ ]:
# Datos:
Sgen = 100 # MVA.
Ugen = 15 # kV.
H = 3 # s.
Xgen = 0.3 # pu (transitoria).
Ured = 220 # kV.
St = 100 # MVA.
Xt = 0.15 # pu.
XlA = 0.20 # pu.
XlB = 0.20 # pu.
dist_falta = 0.5 # Posición de la falta en la línea (distancia relativa desde cabecera).
FPgen = 0.8 # factor de potencia del generador.
Gen = 1 # Régimen de carga del generador (tanto por uno).
f = 50 # Hz.
omega_0 = 2*np.pi*f # Velocidad angular eléctrica de sincronismo (rad/s).

### Transformación a valores por unidad

In [ ]:
# Magnitudes base:
Sbase = 100 # Potencia base (MVA).
Ub1 = Ugen # Tensión base zona 1 (generador) (kV).
Ub2 = Ured # Tensión base zona 2 (red) (kV).

# Magnitudes transformadas:
ugen = Ugen/Ub1 # Tensión del generador.
ured = Ured/Ub2 # Tensión de red.
sgen = Sgen/Sbase # Potencia entregada por el generador pre-falta.
H = H*Sgen/Sbase # Constante de inercia referida a la potencia base del sistema.

### Condiciones pre-falta

In [ ]:
# Corriente inyectada por el generador antes de la falta:
sgen_comp = sgen*Gen*(FPgen+cm.sin(np.arccos(FPgen))*1j) # Potencia compleja entregada por el generador.
igen = sgen_comp.conjugate()/ugen.conjugate() # Corriente inyectada pre-falta.

# Tensión detrás de la reactancia transitoria del generador respecto de los bornes del mismo:
egen = ugen+Xgen*1j*igen
print('E_gen = {:.3} pu.'.format(egen))

# Tensión del nodo de conexión a red respecto de los bornes del generador:
ured = ugen - (Xt + XlA*XlB/(XlA + XlB))*1j*igen
print('U_red = {:.3} pu.'.format(ured))

# Ángulo relativo de la tensión detrás de la reactancia transitoria del generador con relación a la tensión del nudo de potencia infinita ANTES DE LA FALTA:
delta_0 = cm.phase(egen)-cm.phase(ured) # Ángulo de fase en sincronismo [rad].
delta_0_deg = delta_0*180/np.pi # [deg].
print('delta_pref = {:.3} deg.'.format(delta_0_deg))

# Impedancia de transferencia antes de la falta:
Xe_pref = Xgen + Xt + XlA*XlB/(XlA+XlB)
print('Xe_pref = {:.3} pu.'.format(Xe_pref))

# Potencia máxima transmisible antes de la falta:
Pmax_pref = abs(egen)*abs(ured)/Xe_pref
print('Pmax_pref = {:.3} pu.'.format(Pmax_pref))

# Potencia eléctrica:
delta = np.arange(0,np.pi,0.01)
delta_deg = delta*180/np.pi
pelec = abs(egen)*abs(ured)*np.sin(delta)/Xe_pref

# Representación gráfica
anotaciones = True
fig = plt.figure(figsize=(12,12),dpi= 80)
plt.grid() # Malla.
plt.ylabel('Potencia [pu]')
plt.xlabel('Ángulo eléctrico [deg]')

# Curva de potencia eléctrica pre-falta:
fpelec = plt.plot(delta_deg, pelec, label = 'Pelec pre-falta')

# Curva de potencia mecánica:
pmec = sgen_comp.real*np.ones(np.size(delta))
fpmec = plt.plot(delta_deg, pmec, label = 'Pmec')

# Punto de operación antes de la falta:
p1x = delta_0_deg
p1y = sgen_comp.real
p1 = plt.plot([p1x], [p1y], 'bo', markersize=12, label = 'Fto. pre-falta')
## Anotaciones:
if anotaciones:
    plt.text (p1x-3, p1y+0.05, '0', fontsize=14, horizontalalignment='center', verticalalignment='center')

# Leyenda del gráfico:
plt.legend(loc = 'lower right')
plt.show()

### Condiciones en falta

Durante el periodo de duración de la falta y el estado de los interruptores mantenido según las condiciones previas a la falta, puede transmitirse potencia entre el generador y la red a través de la línea A, no afectada por la falta trifásica en el punto **F**. Debe calcularse entonces la **impedancia de transferencia** entre la fuerza electromotriz del generador y el nudo de potencia infinita.

La impedancia de transferencia es la impedancia serie, sin ramas en paralelo, entre los nudos analizados, en este caso los nudos 0 (antes de la impedancia interna del generador) y 3. Para su determinación deberán realizarse las transformaciones necesarias del circuito, en este caso una transformación ESTRELLA > TRIÁNGULO.

In [ ]:
# Impedancia de transferencia en falta:
Xe_f = Xgen + Xt + XlA + (Xgen + Xt)*XlA/(dist_falta*XlB)
print('Xe_f = {:.3} pu.'.format(Xe_f))

# Potencia máxima transmisible en falta:
Pmax_f = abs(egen)*abs(ured)/Xe_f
print('Pmax_f = {:.3} pu.'.format(Pmax_f))

# Potencia eléctrica en falta:
delta = np.arange(0,np.pi,0.01)
delta_deg = delta*180/np.pi
pelecf = abs(egen)*abs(ured)*np.sin(delta)/Xe_f

# Representación gráfica
anotaciones = True
fig = plt.figure(figsize=(12,12),dpi= 80)
plt.grid() # Malla.
plt.ylabel('Potencia [pu]')
plt.xlabel('Ángulo eléctrico [deg]')

# Curva de potencia eléctrica pre-falta:
fpelec = plt.plot(delta_deg, pelec, label = 'Pelec pre-falta')

# Curva de potencia mecánica:
pmec = sgen_comp.real*np.ones(np.size(delta))
fpmec = plt.plot(delta_deg, pmec, label = 'Pmec')

# Punto de operación antes de la falta:
p1x = delta_0_deg
p1y = sgen_comp.real
p1 = plt.plot([p1x], [p1y], 'bo', markersize=12, label = 'Fto. pre-falta')
## Anotaciones:
if anotaciones:
    plt.text (p1x-3, p1y+0.05, '0', fontsize=14, horizontalalignment='center', verticalalignment='center')

# Curva de potencia eléctrica en falta:
fpelecf = plt.plot(delta_deg, pelecf, color='red', label = 'Pelec en falta') 

# Punto de operación en falta:
p2x = delta_0_deg
p2y = abs(egen)*abs(ured)*np.sin(delta_0)/Xe_f
p2 = plt.plot([p2x], [p2y], 'ro', markersize=12, label = 'Fto. en falta')
## Anotaciones:
if anotaciones:
    plt.text (p2x-6, p2y+0.05, '1', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x, p2y), xytext=(p1x, p1y), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))
    
# Leyenda del gráfico:
plt.legend(loc = 'lower right')
plt.show()

Si la nueva **curva característica eléctrica** en falta a la **curva característica mecánica** durante el tiempo de permanencia de la falta nunca podrá sincronizarse de forma natual el generador. En caso contrario, el ángulo eléctrico $\delta$ evolucionaría hasta el punto de corte con la característica mecánica (acumulando energía aceleradora) y  posteriormente deceleraría hasta el punto de máxima oscilación, en el cual, la energía (área) deceleradora igualaría a la energía (área) aceleradora acumulada. En caso de que no se dispusiera de suficiente área deceleradora y no se modificara la curva característica eléctrica, el sistema sería inestable. 

En caso de existir suficiente área deceleradora, el sistema alcanzaría el punto de máxima oscilación y retornaría hasta que, pasado el punto de corte de las curvas características, se acumulara tanta área aceleradora como deceleradora, es decir, retornaría a $\delta_0$ y oscilaría permanentemente entre estos dos puntos de oscilación. En caso de existencia de factor de amortiguamiento no nulo, la amplitud de las oscilaciones dismuniría con el tiempo y el sistema se estabilizaría en el nuevo punto de corte de las curvas características.

### Condiciones post-falta

Detectada la falta por los sistemas de protección, la operación habitual para evitar la existencia de una corriente de cortocircuito de forma permanente en el sistema, sería abrir la línea afectada, en este caso la línea 23B, mediante la apertura de los interruptores de línea (I1B ó I1B y I2B). Hecho esto, el circuito quedaría, según se muestra en la figura, con capacidad de transmisión de potencia a través de la línea 23A, pero con una **nueva impedancia de transferencia** distinta a las correspondientes a las situaciones pre-falta y en falta.

In [ ]:
# Ángulo de despeje de la falta = apertura del interruptor:
delta_desp_deg = 65 # [deg].
delta_desp = delta_desp_deg*np.pi/180 # [rad].

# Impedancia de transferencia después de la falta:
Xe_posf = Xgen + Xt + XlA
print('Xe_posf = {:.3} pu.'.format(Xe_posf))

# Potencia máxima transmisible post-falta:
Pmax_posf = abs(egen)*abs(ured)/Xe_posf
print('Pmax_posf = {:.3} pu.'.format(Pmax_posf))

# Potencia eléctrica post-falta:
delta = np.arange(0,np.pi,0.01)
delta_deg = delta*180/np.pi
pelecposf = abs(egen)*abs(ured)*np.sin(delta)/Xe_posf

# Representación gráfica
anotaciones = True
fig = plt.figure(figsize=(12,12),dpi= 80)
plt.grid() # Malla.
plt.ylabel('Potencia [pu]')
plt.xlabel('Ángulo eléctrico [deg]')

# Curva de potencia eléctrica pre-falta:
fpelec = plt.plot(delta_deg, pelec, label = 'Pelec pre-falta')

# Curva de potencia mecánica:
pmec = sgen_comp.real*np.ones(np.size(delta))
fpmec = plt.plot(delta_deg, pmec, label = 'Pmec')

# Punto de operación antes de la falta:
p1x = delta_0_deg
p1y = sgen_comp.real
p1 = plt.plot([p1x], [p1y], 'bo', markersize=12, label = 'Fto. pre-falta')
## Anotaciones:
if anotaciones:
    plt.text (p1x-3, p1y+0.05, '0', fontsize=14, horizontalalignment='center', verticalalignment='center')

# Curva de potencia eléctrica en falta:
fpelecf = plt.plot(delta_deg, pelecf, color='red',label = 'Pelec en falta') 

# Punto de operación en falta:
p2x = np.array([delta_0_deg, delta_desp_deg])
p2y = np.array([abs(egen)*abs(ured)*np.sin(delta_0)/Xe_f, abs(egen)*abs(ured)*np.sin(delta_desp)/Xe_f])
p2 = plt.plot(p2x, p2y, 'ro', markersize=12, label = 'Fto. en falta')
## Anotaciones:
if anotaciones:
    plt.text (p2x[0]-6, p2y[0]+0.05, '1', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x[0], p2y[0]), xytext=(p1x, p1y), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))
    plt.text (p2x[1]-6, p2y[1]+0.05, '2', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x[1], p2y[1]), xytext=(p2x[0], p2y[0]), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))

# Curva de potencia eléctrica post-falta:
fpelecposf = plt.plot(delta_deg, pelecposf, label = 'Pelec post-falta') 

# Punto de operación post-falta:
p3x = delta_desp_deg
p3y = abs(egen)*abs(ured)*np.sin(delta_desp)/Xe_posf
p3 = plt.plot([p3x], [p3y], 'go', markersize=12, label = 'Fto. post-falta')
## Anotaciones:
if anotaciones:
    plt.text (p3x-6, p3y+0.05, '3', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p3x, p3y), xytext=(p2x[1], p2y[1]), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=-0.2"))
 
# Leyenda del gráfico:
plt.legend(loc = 'lower right')
plt.show()

### Áreas aceleradora y deceleradora máxima

En este punto téngase en cuenta que, al ser la curva característica eléctrica pre-falta distinta de la post-falta, el ángulo máximo para el área deceleradora está marcado por la nueva curva, pudiéndose obtener mediante:

$$\delta_{max}=\pi-\arcsin\left(\displaystyle\frac{P_{mec}Xe_{posf}}{E\cdot U}\right).$$

In [ ]:
# Ángulo máximo para el área deceleradora:
delta_0_posf = np.arcsin(sgen_comp.real*Xe_posf/(abs(egen)*abs(ured))) # [rad].
delta_0_posf_deg = delta_0_posf*180/np.pi # [deg].
delta_max = np.pi-delta_0_posf # [rad].
delta_max_deg = delta_max*180/np.pi # [deg].

# Cálculo del área aceleradora:
## Potencia aceleradora en función del ángulo eléctrico:
def Pacel(delta,egen,ured,Xe_f,sgen_comp):
    pacel = sgen_comp.real-abs(egen)*abs(ured)*np.sin(delta)/Xe_f
    return pacel

## Área aceleradora:
def Aacel(delta_1,delta_2):
    aacel = integrate.quad(Pacel, delta_1, delta_2, args=(egen,ured,Xe_f,sgen_comp))
    return aacel[0]

Area_acel = Aacel(delta_0, delta_desp)
print('Área aceleradora: {:.4} pu.'.format(Area_acel))

# Cálculo del área deceleradora máxima:
## Potencia deceleradora en función del ángulo eléctrico:
def Pdecel(delta,egen,ured,Xe_posf,sgen_comp):
    pdecel = abs(egen)*abs(ured)*np.sin(delta)/Xe_posf-sgen_comp.real
    return pdecel

## Área deceleradora máxima:
def Adecel(delta_1,delta_2):
    adecel = integrate.quad(Pdecel, delta_1, delta_2, args=(egen,ured,Xe_posf,sgen_comp))
    return adecel[0]

Area_decel_max = Adecel(delta_desp, delta_max)
print('Área deceleradora máxima: {:.4} pu.'.format(Area_decel_max))

In [ ]:
# Representación gráfica
anotaciones = True
fig = plt.figure(figsize=(12,12),dpi= 80)
plt.grid() # Malla.
plt.ylabel('Potencia [pu]')
plt.xlabel('Ángulo eléctrico [deg]')

# Curva de potencia eléctrica pre-falta:
fpelec = plt.plot(delta_deg, pelec, label = 'Pelec pre-falta')

# Curva de potencia mecánica:
pmec = sgen_comp.real*np.ones(np.size(delta))
fpmec = plt.plot(delta_deg, pmec, label = 'Pmec')

# Punto de operación antes de la falta:
p1x = delta_0_deg
p1y = sgen_comp.real
p1 = plt.plot([p1x], [p1y], 'bo', markersize=12, label = 'Fto. pre-falta')
## Anotaciones:
if anotaciones:
    plt.text (p1x-3, p1y+0.05, '0', fontsize=14, horizontalalignment='center', verticalalignment='center')

# Curva de potencia eléctrica en falta:
fpelecf = plt.plot(delta_deg, pelecf, color='red', label = 'Pelec en falta') 

# Punto de operación en falta:
p2x = np.array([delta_0_deg, delta_desp_deg])
p2y = np.array([abs(egen)*abs(ured)*np.sin(delta_0)/Xe_f, abs(egen)*abs(ured)*np.sin(delta_desp)/Xe_f])
p2 = plt.plot(p2x, p2y, 'ro', markersize=12, label = 'Fto. en falta')
## Anotaciones:
if anotaciones:
    plt.text (p2x[0]-6, p2y[0]+0.05, '1', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x[0], p2y[0]), xytext=(p1x, p1y), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))
    plt.text (p2x[1]-6, p2y[1]+0.05, '2', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x[1], p2y[1]), xytext=(p2x[0], p2y[0]), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))

# Curva de potencia eléctrica post-falta:
fpelecposf = plt.plot(delta_deg, pelecposf, label = 'Pelec post-falta') 

# Punto de operación post-falta:
p3x = delta_desp_deg
p3y = abs(egen)*abs(ured)*np.sin(delta_desp)/Xe_posf
p3 = plt.plot([p3x], [p3y], 'go', markersize=12, label = 'Fto. post-falta')
## Anotaciones:
if anotaciones:
    plt.text (p3x-6, p3y+0.05, '3', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p3x, p3y), xytext=(p2x[1], p2y[1]), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=-0.2"))

# Área aceleradora:
delta_deg2 = np.ma.masked_greater(delta_deg,p2x[1])
plt.fill_between(delta_deg2, pelecf, pmec, where = pelec >= pmec, facecolor='green', interpolate=True, label='Área aceleradora')

# Área deceleradora máxima:
delta_deg3 = np.ma.masked_greater(delta_deg, delta_max_deg)
plt.fill_between(delta_deg3, pmec, pelecposf, where = delta_deg3 >= p2x[1], facecolor='red', interpolate=True, label='Área deceleradora máx.')

# Leyenda del gráfico:
plt.legend(loc = 'lower right')
plt.show()

Al ser el área aceleradora menor que el área deceleradora, el sistema podrá alcanzar la estabilidad y el desplazamiento máximo del ángulo será menor que el ángulo máximo:

$$\delta_{max}\leq \pi-\delta_{0,posf}.$$

### Ángulo de oscilación máximo

In [ ]:
# Función del residuo de la diferencia de áreas aceleradora y deceleradora en función del ángulo de reconexión delta:
def IgualdadAreas(delta):
    return (Aacel(delta_0,delta_desp)-Adecel(delta_desp,delta))**2

# Comenzamos la optimización: función residuo de la diferencia de áreas mínima:
delta_start = (delta_0+delta_max)/2 # Valor semilla.
bnd_x1 = 1.001*delta_0 # Límite inferior (restricción).
bnd_x2 = delta_max # Límite superior (restricción).
limites = optim.Bounds(bnd_x1,bnd_x2)
delta_opt = optim.minimize(IgualdadAreas,np.array([delta_start]),method='L-BFGS-B',bounds=limites).x
delta_1 = delta_opt[0] # [rad].
delta_1_deg = delta_1*180/np.pi # [deg].
print('El ángulo de oscilación máximo es: {:.3f} deg'.format(delta_1_deg))

In [ ]:
# Representación gráfica
anotaciones = True
fig = plt.figure(figsize=(12,12),dpi= 80)
plt.grid() # Malla.
plt.ylabel('Potencia [pu]')
plt.xlabel('Ángulo eléctrico [deg]')

# Curva de potencia eléctrica pre-falta:
fpelec = plt.plot(delta_deg, pelec, label = 'Pelec pre-falta')

# Curva de potencia mecánica:
pmec = sgen_comp.real*np.ones(np.size(delta))
fpmec = plt.plot(delta_deg, pmec, label = 'Pmec')

# Punto de operación antes de la falta:
p1x = delta_0_deg
p1y = sgen_comp.real
p1 = plt.plot([p1x], [p1y], 'bo', markersize=12, label = 'Fto. pre-falta')
## Anotaciones:
if anotaciones:
    plt.text (p1x-3, p1y+0.05, '0', fontsize=14, horizontalalignment='center', verticalalignment='center')

# Curva de potencia eléctrica en falta:
fpelecf = plt.plot(delta_deg, pelecf, color='red', label = 'Pelec en falta') 

# Punto de operación en falta:
p2x = np.array([delta_0_deg, delta_desp_deg])
p2y = np.array([abs(egen)*abs(ured)*np.sin(delta_0)/Xe_f, abs(egen)*abs(ured)*np.sin(delta_desp)/Xe_f])
p2 = plt.plot(p2x, p2y, 'ro', markersize=12, label = 'Fto. en falta')
## Anotaciones:
if anotaciones:
    plt.text (p2x[0]-6, p2y[0]+0.05, '1', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x[0], p2y[0]), xytext=(p1x, p1y), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))
    plt.text (p2x[1]-6, p2y[1]+0.05, '2', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x[1], p2y[1]), xytext=(p2x[0], p2y[0]), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))

# Curva de potencia eléctrica post-falta:
fpelecposf = plt.plot(delta_deg, pelecposf, label = 'Pelec post-falta') 

# Punto de operación post-falta:
p3x = delta_desp_deg
p3y = abs(egen)*abs(ured)*np.sin(delta_desp)/Xe_posf
p3 = plt.plot([p3x], [p3y], 'go', markersize=12, label = 'Fto. post-falta')
## Anotaciones:
if anotaciones:
    plt.text (p3x-6, p3y+0.05, '3', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p3x, p3y), xytext=(p2x[1], p2y[1]), size=20, va="center", ha="center", arrowprops=dict(facecolor='blue',arrowstyle="simple",connectionstyle="arc3,rad=-0.2"))

# Punto de oscilación máxima:
p4x = delta_1_deg
p4y = abs(egen)*abs(ured)*np.sin(delta_1)/Xe_posf
p4 = plt.plot([p4x], [p4y], 'yo', markersize=12, label = 'Pto. oscilación máximo')
## Anotaciones:
if anotaciones:
    plt.text (p4x-6, p4y+0.05, '4', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p4x, p4y), xytext=(p3x, p3y), size=20, va="center", ha="center", arrowprops=dict(facecolor='blue',arrowstyle="simple",connectionstyle="arc3,rad=-0.2"))
    
# Área aceleradora:
delta_deg2 = np.ma.masked_greater(delta_deg,p2x[1])
plt.fill_between(delta_deg2, pelecf, pmec, where = pelec >= pmec, facecolor='green', interpolate=True, label='Área aceleradora')

# Área deceleradora:
delta_deg3 = np.ma.masked_greater(delta_deg, delta_1_deg)
plt.fill_between(delta_deg3, pmec, pelecposf, where = delta_deg3 >= p2x[1], facecolor='red', interpolate=True, label='Área deceleradora máx.')

# Leyenda del gráfico:
plt.legend(loc = 'lower right')
plt.show()

### Cálculo del ángulo crítico

In [ ]:
## Comenzamos la optimización: función residuo de la diferencia de áreas mínima:
delta_start = (delta_0+delta_max)/2 # Valor semilla.
bnd_x1 = delta_0 # Límite inferior (restricción).
bnd_x2 = delta_max # Límite superior (restricción).
limites = optim.Bounds(bnd_x1,bnd_x2)
delta_opt = optim.minimize(IgualdadAreas,np.array([delta_start]),method='L-BFGS-B',bounds=limites).x
delta_cri = delta_opt[0] # rad.
delta_cri_deg = delta_cri*180/np.pi # deg.
print('El ángulo crítico calculado es: {:.3f} deg'.format(delta_cri_deg))

### Oscilación debida al nuevo desequilibrio Pelec > Pmec

Salvo que se despejara la falta justo en el ángulo crítico y, por lo tanto, el ángulo de oscilación máximo fuera $\delta_{max}=\pi-\delta_{0,posf}$, caso en el que se encontraría un nuevo equilibrio al cumplirse en dicho punto de funcionamiento que $P_{elec}=P_{mec}$, el sistema aunque haya alcanzado la oscilación máxima (igualdad de áreas = igualdad de energía de aceleración y deceleración), estaría en desequilibrio ($P_{elec}>P_{mec}$) por lo que volvería a oscilar en sentido inverso.

In [ ]:
# Cálculo del área deceleradora:
Area_decel_2 = Adecel(delta_0, delta_1)
print('Área deceleradora 2: {:.4} pu.'.format(Area_decel_2))

# Potencia aceleradora en función del ángulo eléctrico (Pelec != 0):
def Pacel2(delta,sgen_comp,egen,ured,Xe_posf):
    pacel = sgen_comp.real-abs(egen)*abs(ured)*np.sin(delta)/Xe_posf
    return pacel

## Área aceleradora 2:
def Aacel2(delta_1,delta_2):
    aacel2 = integrate.quad(Pacel2, delta_1, delta_2, args=(sgen_comp,egen,ured,Xe_posf))
    return aacel2[0]

# Función del residuo de la diferencia de áreas aceleradora y deceleradora en función del ángulo delta:
def IgualdadAreas3(delta):
    return (Aacel2(delta,delta_0)-Adecel(delta_0,delta_1))**2

# Comenzamos la optimización: función residuo de la diferencia de áreas mínima:
delta_start = (delta_0-delta_max)/2 # Valor semilla.
bnd_x1 = -delta_max # Límite inferior (restricción).
bnd_x2 = 0.99*delta_0 # Límite superior (restricción).
limites = optim.Bounds(bnd_x1,bnd_x2)
delta_opt2 = optim.minimize(IgualdadAreas3,np.array([delta_start]),method='L-BFGS-B',bounds=limites).x
delta_2 = delta_opt2[0] # [rad].
delta_2_deg = delta_2*180/np.pi # [deg].
print('El ángulo de oscilación máximo 2 es: {:.4f} deg'.format(delta_2_deg))

In [ ]:
# Representación gráfica
anotaciones = True
fig = plt.figure(figsize=(12,12),dpi= 80)
plt.grid() # Malla.
plt.ylabel('Potencia [pu]')
plt.xlabel('Ángulo eléctrico [deg]')

# Curva de potencia eléctrica pre-falta:
fpelec = plt.plot(delta_deg, pelec, label = 'Pelec pre-falta')

# Curva de potencia mecánica:
delta2 = np.arange(-1*np.pi,np.pi,0.01)
delta2_deg = delta2*180/np.pi
pmec = sgen_comp.real*np.ones(np.size(delta2))
fpmec = plt.plot(delta2_deg, pmec, label = 'Pmec')

# Punto de operación antes de la falta:
p1x = delta_0_deg
p1y = sgen_comp.real
p1 = plt.plot([p1x], [p1y], 'bo', markersize=12, label = 'Fto. pre-falta')
## Anotaciones:
if anotaciones:
    plt.text (p1x-3, p1y+0.05, '0', fontsize=14, horizontalalignment='center', verticalalignment='center')

# Curva de potencia eléctrica en falta:
fpelecf = plt.plot(delta_deg, pelecf, color='red', label = 'Pelec en falta') 

# Punto de operación en falta:
p2x = np.array([delta_0_deg, delta_desp_deg])
p2y = np.array([abs(egen)*abs(ured)*np.sin(delta_0)/Xe_f, abs(egen)*abs(ured)*np.sin(delta_desp)/Xe_f])
p2 = plt.plot(p2x, p2y, 'ro', markersize=12, label = 'Fto. en falta')
## Anotaciones:
if anotaciones:
    plt.text (p2x[0]-6, p2y[0]+0.05, '1', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x[0], p2y[0]), xytext=(p1x, p1y), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))
    plt.text (p2x[1]-6, p2y[1]+0.05, '2', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p2x[1], p2y[1]), xytext=(p2x[0], p2y[0]), size=20, va="center", ha="center", arrowprops=dict(facecolor='red',arrowstyle="simple",connectionstyle="arc3,rad=0.2"))

# Curva de potencia eléctrica post-falta:
pelecposf = abs(egen)*abs(ured)*np.sin(delta2)/Xe_posf
fpelecposf = plt.plot(delta2_deg, pelecposf, label = 'Pelec post-falta') 

# Punto de operación post-falta:
p3x = delta_desp_deg
p3y = abs(egen)*abs(ured)*np.sin(delta_desp)/Xe_posf
p3 = plt.plot([p3x], [p3y], 'go', markersize=12, label = 'Fto. post-falta')
## Anotaciones:
if anotaciones:
    plt.text (p3x-6, p3y+0.05, '3', fontsize=14, horizontalalignment='center', verticalalignment='center')
    plt.annotate("", xy=(p3x, p3y), xytext=(p2x[1], p2y[1]), size=20, va="center", ha="center", arrowprops=dict(facecolor='blue',arrowstyle="simple",connectionstyle="arc3,rad=-0.2"))

# Punto de oscilación máximo 1:
p4x = delta_1_deg
p4y = abs(egen)*abs(ured)*np.sin(delta_1)/Xe_posf
p4 = plt.plot([p4x], [p4y], 'yo', markersize=12, label = 'Pto. oscilación máximo 1')
## Anotaciones:
if anotaciones:
    plt.text (p4x+3, p4y+0.1, '4=6=...', fontsize=14, horizontalalignment='left', verticalalignment='center')
    plt.annotate("", xy=(p4x, p4y), xytext=(p3x, p3y), size=20, va="center", ha="center", arrowprops=dict(facecolor='blue',arrowstyle="simple",connectionstyle="arc3,rad=-0.2"))

# Punto de oscilación máximo 2:
p5x = delta_2_deg
p5y = abs(egen)*abs(ured)*np.sin(delta_2)/Xe_posf
p5 = plt.plot([p5x], [p5y], 'mo', markersize=12, label = 'Pto. oscilación máximo 2')
## Anotaciones:
if anotaciones:
    plt.text (p5x+3, p5y-0.1, '5=7=...', fontsize=14, horizontalalignment='left', verticalalignment='center')
    plt.annotate("", xy=(p5x, p5y), xytext=(p4x, p4y), size=20, va="center", ha="center", arrowprops=dict(facecolor='green',arrowstyle="simple",connectionstyle="arc3,rad=-0.5"))
    plt.annotate("", xy=(p4x, p4y), xytext=(p5x, p5y), size=20, va="center", ha="center", arrowprops=dict(facecolor='green',arrowstyle="simple",connectionstyle="arc3,rad=-0.5"))

# Área deceleradora:
delta_deg2 = np.ma.masked_less(delta2_deg, delta_0_posf_deg)
delta_deg2 = np.ma.masked_greater(delta_deg2, delta_1_deg)
plt.fill_between(delta_deg2, pmec, pelecposf, where = pelecposf >= pmec, facecolor='red', interpolate=True, label='Área deceleradora')

# Área aceleradora:
delta_deg3 = np.ma.masked_less(delta2_deg,delta_2_deg)
delta_deg3 = np.ma.masked_greater(delta_deg3,delta_0_posf_deg)
plt.fill_between(delta_deg3, pelecposf, pmec, where = pelecposf <= pmec, facecolor='green', interpolate=True, label='Área aceleradora')

# Leyenda del gráfico:
plt.legend(loc = 'lower right')
plt.show()

Tras alcanzar el punto de oscilación máximo 2, la energía de las áreas se habría compensado pero el sistema estaría de nuevo en desequilibrio al ser $P_{elec} < P_{mec}$, por lo que el sistema retrocedería de nuevo, hasta el punto de oscilación máximo 1.

En ausencia de **amortiguamiento** el sistema se encontrará oscilando entre los puntos de oscilación máximos 1 y 2 calculados. Si exisitiera amortiguamiento (factor de amortiguamiento $D$ no nulo), las oscilaciones se irían reduciendo hasta converger al punto de equilibrio original (suponiendo que $P_{mec}$ se mantuviera). **Obsérvese que el nuevo punto de equilibrio que se alcanzaría sería distinto del original al encontrarse en una curva característica nueva (característica post-falta)**.

Obsérvese que valores negativos de $P_{elec}$ implicarían que el generador estaría consumiendo potencia activa, en lugar de cederla (para conseguir frenarse, el generador estaría actuando como motor de la turbina).

### Límites de frecuencia alcanzados

De acuerdo a los puntos máximos de oscilación el generador alcanzará una frecuencia máxima/sobrefrecuencia y una frecuencia mínima/subfrecuencia. Obsérvese que estos puntos extremos se alcanzan cuando el sistema pasa por el ángulo $\delta_0$, mientras que la frecuencia en los puntos de oscilación 1 y 2, es la frecuencia original del sistema ($50\mbox{ Hz}$).

In [ ]:
# Frecuencia original:
frec_0 = omega_0/(2*np.pi)
print('Frecuencia origen: {:.4f} Hz.'.format(frec_0))

# Frecuencia en el punto de reconexión:
## Potencia aceleradora en función del ángulo eléctrico:
def Pacel(delta,egen,ured,Xe_f,sgen_comp):
    pacel = sgen_comp.real-abs(egen)*abs(ured)*np.sin(delta)/Xe_f
    return pacel

## Área aceleradora:
def Aacel(delta_1,delta_2):
    aacel = integrate.quad(Pacel, delta_1, delta_2, args=(egen,ured,Xe_f,sgen_comp))
    return aacel[0]

omega_desp = omega_0+((omega_0/2*H)*Aacel(delta_0, delta_desp))**0.5
frec_desp = omega_desp/(2*np.pi)
print('Frecuencia alcanzada en el punto de reconexión del generador: {:.4f} Hz.'.format(frec_desp))

# Frecuencia en el punto de oscilación máximo 1:
omega_1 = omega_desp-((omega_0/2*H)*Adecel(delta_desp, delta_1))**0.5
frec_1 = omega_1/(2*np.pi)
print('Frecuencia alcanzada en el punto de oscilación máximo 1: {:.4f} Hz.'.format(frec_1))

# Frecuencia en el punto origen:
omega_2 = omega_1-((omega_0/2*H)*Adecel(delta_0, delta_1))**0.5
frec_2 = omega_2/(2*np.pi)
print('Frecuencia alcanzada al pasar de osc. máx. 1 a osc. máx. 2 por el punto origen: {:.4f} Hz.'.format(frec_2))

# Frecuencia en el punto de oscilación máximo 2:
omega_3 = omega_2+((omega_0/2*H)*Aacel2(delta_2, delta_0))**0.5
frec_3 = omega_3/(2*np.pi)
print('Frecuencia alcanzada en el punto de oscilación máximo 2: {:.4f} Hz.'.format(frec_3))

# Frecuencia en el punto origen:
omega_4 = omega_3+((omega_0/2*H)*Aacel2(delta_2, delta_0))**0.5
frec_4 = omega_4/(2*np.pi)
print('Frecuencia alcanzada al pasar de osc. máx. 2 a osc. máx. 1 por el punto origen: {:.4f} Hz.'.format(frec_4))

# Frecuencia en el punto de oscilación máximo 1:
omega_5 = omega_4-((omega_0/2*H)*Adecel(delta_0, delta_1))**0.5
frec_5 = omega_5/(2*np.pi)
print('Frecuencia alcanzada en el punto de oscilación máximo 1: {:.4f} Hz.'.format(frec_5))